In [27]:
import ast

class sgrepVisitor(ast.NodeVisitor):
    def __init__(self):
        self.scope_stack = [] #tracks the current path (e.g ['Database', 'connect'])
        self.chunks = [] #stores the results

    def _get_context_header(self):
        return " > ".join(self.scope_stack)

    def visit_ClassDef(self, node):
        self.scope_stack.append(f"Class: {node.name}")
        self.generic_visit(node)
        self.scope_stack.pop()

    def visit_FunctionDef(self, node):
        self.scope_stack.append(f"Function: {node.name}")
        func_source = ast.get_source_segment(self.source_code, node)
        header = self._get_context_header()
        self.chunks.append({
            "context": header,
            "code": func_source,
            "line_start": node.lineno
        })
        self.generic_visit(node)
        self.scope_stack.pop()

    def parse(self, source_code):
        self.source_code = source_code
        tree = ast.parse(source_code)
        self.visit(tree)
        return self.chunks


f = open("sample_dir/sample.py")
sample_code = f.read()
f.close()

visitor = sgrepVisitor()
parsed_chunks = visitor.parse(sample_code)

for chunk in parsed_chunks:
    print(f"[{chunk['context']}]")
    print(chunk['code'])
    print("="*40)

[Class: Database > Function: __init__]
def __init__(self, connection_string):
        self.conn = connection_string
[Class: Database > Function: connect]
def connect(self):
        print("Connecting...")

        def retry():
            print("Retrying connection...")
            return True

        return retry()
[Class: Database > Function: connect > Function: retry]
def retry():
            print("Retrying connection...")
            return True
[Function: standalone_function]
def standalone_function():
    return "I have no class"
